In [26]:
import cv2
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from ultralytics import YOLO


In [ ]:
# Load pre-trained ResNet model
resnet = models.resnet50(pretrained=True)
resnet.eval()  # Set to evaluation mode

: 

In [28]:
# Load the trained YOLOv8 model
model_yolo = YOLO('file_path')

In [29]:
# Define image transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
def extract_features(image, model):
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        features = model(image)
    return features.numpy().flatten()

# Function to perform product detection using YOLO and clustering using K-means
def detect_and_classify_products(image_path, n_clusters=5):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect products using YOLOv8
    results = model_yolo.predict(image_path)
    boxes = results[0].boxes.xyxy.cpu().numpy()  # Extract bounding boxes

    # Extract features from each detected product and store them
    features = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box)
        product_region = image_rgb[y1:y2, x1:x2]
        product_features = extract_features(product_region, resnet)
        features.append(product_features)

    # Perform PCA for dimensionality reduction
    pca = PCA(n_components=20)  # Reduce features to 20 dimensions
    reduced_features = pca.fit_transform(features)

    # Perform K-means clustering on the features
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(reduced_features)
    labels = kmeans.labels_

    # Plot clustered products with different colors
    for idx, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        label = labels[idx]
        color = (int(label * 50 % 255), int(label * 100 % 255), int(label * 150 % 255))
        cv2.rectangle(image_rgb, (x1, y1), (x2, y2), color, 3)

    # Show clustered image
    plt.imshow(image_rgb)
    plt.axis('off')
    plt.title('Product Classification and Clustering')
    plt.show()

# Example usage
image_path = 'file_path'
detect_and_classify_products(image_path, n_clusters=5)

: 